# W266 Final Project: Nautral Language Processing on Wine Reviews 
Maria Corina Cabezas, Austin Doolitle

## Background

In [9]:
#add introduction

## Prerequisite Libraries

In [26]:
import random
import math
import collections
import string

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.preprocessing import Imputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from yellowbrick.regressor import PredictionError
from xgboost.sklearn import XGBRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import keras

seed = 42
random.seed(seed)
np.random.seed(42)

[nltk_data] Downloading package punkt to /home/austin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/austin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data Exploration and Analysis

### Data Cleaning

In [2]:
# Read the data
df = pd.read_csv('./data/winemag-data-130k-v2.csv', index_col=0)
print(f'N Rows: {len(df.index)}')
df.head()

N Rows: 129971


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


The dataset contains a lot of duplicates. I noticed that simply running data.drop_duplicates() did not remove all of them, because some column values were slightly different. Nevertheless, the fact that the "Description" columns were identical was sufficient to determine these were not different reviews. I decided to drop all duplicates based on the description column alone.

In [3]:
df = df.drop_duplicates('description')
print(f'N Rows: {len(df.index)}')

N Rows: 119955


Let's look at the number of missing values on each column

In [4]:
df.isnull().sum(axis = 0)

country                     59
description                  0
designation              34532
points                       0
price                     8388
province                    59
region_1                 19558
region_2                 73195
taster_name              24912
taster_twitter_handle    29441
title                        0
variety                      1
winery                       0
dtype: int64

It makes sense to delete the columns that have a very large number of missing values like designation, region_2 and taster_twitter_handle

In [5]:
columns = ['designation','region_2','taster_twitter_handle']
df = df.drop(columns, axis=1)

We also noticed that price has 8388 missing values, so we will replace these with the average price of wine

In [6]:
# TODO Imputing is mostly meant for features, not for the target values.
# we should probably just drop missing values

imp=Imputer(missing_values="NaN", strategy="mean" )
df["price"]=imp.fit_transform(df[["price"]]).ravel()

/opt/miniconda3/envs/jupyter/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


## Train/Test/Dev Split

 First we split our dataset into 3 sets, namely training set, dev set, and test set. This is so that we can be more confident at our models and to better compare the model options we come up with. Specifically, training set is used to train our models, dev set is used to optimize each model, and test set is used to evaluate performance of the model. We have assigned 20% of the total dataset to be the test set, 20% to be the dev set, and the rest being training set. We made the split here because it will be used in the Data Exploration.

In [7]:
# split it into train, dev, and test
# courtesy of https://stackoverflow.com/a/38251063
perm = np.random.permutation(df.index)
m = len(df.index)

train_percent = .6
dev_percent = .2

train_end = int(m * train_percent)
dev_end = int(m * dev_percent) + train_end

train_df = df.loc[perm[:train_end]]
dev_df = df.loc[perm[train_end:dev_end]]
test_df = df.loc[perm[dev_end:]]

print(f'Train shape: {train_df.shape}')
print(f'Dev shape: {dev_df.shape}')
print(f'Test shape: {test_df.shape}')

Train shape: (71973, 10)
Dev shape: (23991, 10)
Test shape: (23991, 10)


## Initial Investigation
Let's start by just looking at the data and seeing what pops out to us. We'll spend a little time looking at the continuous values points and price, and then spend more time looking at the words

In [25]:
train_df.describe(include='all')

,country,description,points,price,province,region_1,taster_name,title,variety,winery
count,71934,71973,71973.000000,71973.000000,71934,60248,56995,71973,71972,71973
unique,41,71973,NaN,NaN,387,1135,19,71515,627,14262
top,US,"Big but polished, this full-bodied wine almost...",NaN,NaN,California,Napa Valley,Roger Voss,Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma...,Pinot Noir,Wines & Winemakers
freq,30183,1,NaN,NaN,20167,2536,14186,8,7319,122
mean,NaN,NaN,88.444625,35.712472,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,3.094782,40.882064,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,80.000000,4.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,86.000000,18.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,88.000000,28.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,91.000000,40.000000,NaN,NaN,NaN,NaN,NaN,NaN


### Points

It appears that the points are located between 80-100. This matches up with the source of the data that claims they do not publish reviews for any wine scored less than 80. More information on wine scoring can be viewed [here](https://www.winespectator.com/articles/scoring-scale)

In [ ]:
_ = sns.distplot(train_df.points, bins=21, kde_kws={'bw':1})

So it looks like this is a textbook normal distribution with an incredibly slight left skew. Let's just make sure that every value is represented.

In [ ]:
print(np.sort(train_df.points.unique()))

### Price

Since this is related to currency, I bet my lunch this is a heavy tailed distribution. Let's see.

In [ ]:
print(f'Missing values: {train_df.price.isna().sum()}')

# let's remove the missing prices for now
sns.distplot(train_df[~train_df.price.isna()].price)

Nice... lets log scale this.

In [ ]:
train_df['price_log'] = np.log(train_df.price)
ax = sns.distplot(train_df[~train_df.price_log.isna()].price_log, bins=100)

Just out of curiosity, I see that the lowest price is 4 dollars... what's the score on those?

In [ ]:
train_df[train_df.price == 4].points.mean()

### String Columns

In [ ]:
non_cont_columns = train_df.columns[train_df.dtypes == np.object]
print(f'Non-Continuous columns: {list(non_cont_columns)}')

Let's start by focusing on the descriptions. We'll tokenize, canonize, construct a vocabulary, and finally get counts for each of the items.

In [8]:
counter = collections.Counter()

for s in train_df.description:
    s = s.lower()
    tokenized = nltk.tokenize.word_tokenize(s)
    counter.update(tokenized)

vocab = counter.items()
print(len(vocab))

36322


In [ ]:
# view most common words 
most_common = counter.most_common(50)
print(most_common)

One very interesting thing viewed here is that in the most common words are very wine specific words. This is pretty obvious in hindsight, but it made me laugh to see wine just outside of the top 10.

Now we would like to see a wordcloud view of the most common words excluding punctuations and stopwords. We also excluded other common words like drink, wine, flavors etc.

In [ ]:
text = " ".join(review for review in df.description)
stopwords = set(STOPWORDS)
stopwords.update(["drink", "now", "wine", "flavor", "flavors"])

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
plt.figure(figsize=[20,20])
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

It seems black cherry and full bodied are the most common characteristics and Carbernet Sauvignon is the most discussed type of wine. Let's see if this is due to it being more common in the dataset

In [ ]:
variety_df = df.groupby('variety').filter(lambda x: len(x) > 2500)
varieties = variety_df['variety'].value_counts().index.tolist()
fig, ax = plt.subplots(figsize = (6, 4))
sns.countplot(x = variety_df['variety'], order = varieties, ax = ax)
plt.xticks(rotation = 90)
plt.show()


Surprisingly, Cabernet Sauvignon was the third most reviewed wine variety after Pinot Noir and Chardonnay.

### Tasters
Now let's look at the tasters. We'll see how many there are, and the distribution of the number of reviews each has given

In [ ]:
tasters = train_df.taster_name.dropna().unique()
print(f'{len(tasters)} Tasters: {list(tasters)}')
train_df.taster_name = train_df.taster_name.astype('category')

taster_counts  = train_df.taster_name.value_counts()
ax = sns.countplot(train_df.taster_name, order=taster_counts.index)
_ = ax.set_xticklabels(taster_counts.index, rotation=90)

As expected, there appears to be a Zeta distribution in the contributions of each taster. Let's what the numbers are on the 3 least active contributors: Carrie Dykes, Fiona Adams, and Christina Pickard.

In [ ]:
print(taster_counts[['Carrie Dykes', 'Fiona Adams', 'Christina Pickard']])

We'll need to keep this in mind for subsequent analysis since the data related to their contributions will likely not be as representative as someone on the center or left of the distribution

#### Taster Vocabulary
Just for funsies, let's look at the vocabulary of each individual taster

In [ ]:
taster_vocabs = {n: collections.Counter() for n in tasters}

for _, row in train_df[~train_df.taster_name.isna()].iterrows():
    taster = row.taster_name
    s = row.description
    s = s.lower()
    tokenized = nltk.tokenize.word_tokenize(s)
    taster_vocabs[taster].update(tokenized)

In [ ]:
taster_vocab_lens = {k: len(v.items()) for k, v in taster_vocabs.items()}
taster_vocab_lens = pd.Series(taster_vocab_lens).sort_values().iloc[::-1]

ax = sns.barplot(x=taster_vocab_lens.index, y=taster_vocab_lens)
_ = ax.set_xticklabels(taster_vocab_lens.index, rotation=90)

As expected, Roger Voss is up top since he has had the most opportunity to use unique words. Let's compare the two lists.

In [ ]:
compare_list = []

contribution_count_list = list(taster_counts.index)

print('Vocab Size compared to Contribution:')
for vocab_idx, taster in enumerate(taster_vocab_lens.index):
    count_idx = contribution_count_list.index(taster)
    diff = count_idx - vocab_idx
    print(f'\t{vocab_idx + 1}: {taster} ({diff:+d})')

Kerin O'Keefe appears to be relatively bland in her word usage. 

#### Taster Score Distribution
Let's look at each reviewer's score distribution to make sure we don't have any biased reviewers

In [ ]:
cols = 4
rows = math.ceil(len(tasters) / float(cols))
figs, axes = plt.subplots(rows, cols, figsize=(20,20))
axes = axes.flatten()
for taster, ax in zip(tasters, axes):
    taster_reviews = train_df[train_df.taster_name == taster]
    ax = sns.distplot(taster_reviews.points, ax=ax, bins=20)
    ax.set_title(taster)

plt.tight_layout()

There isn't any bias immediately apparent for any of the reviewers, although this is difficult to ascertain due to the large range in the number of reviews per taster.

#### Taster Price Distribution

And now let's look at the distribution of the log price for each reviewer

In [ ]:
cols = 4
rows = math.ceil(len(tasters) / float(cols))
figs, axes = plt.subplots(rows, cols, figsize=(20,20))
axes = axes.flatten()
for taster, ax in zip(tasters, axes):
    taster_reviews = train_df[(train_df.taster_name == taster) & ~train_df.price_log.isna()]
    ax = sns.distplot(taster_reviews.price_log, ax=ax, bins=20)
    ax.set_title(taster)

plt.tight_layout()

### Unique Words
Let's look at the top 3 unique words used by the tasters. We'll do this by iteratively constructing of the top 3 words used by each taster and removing any duplicate words across top 3 lists.

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')

# collect the top 25 words from the corpus (not including nltk stop words)
top_domain_specific_words = []
n = 25
for word, count in counter.most_common():
    if word in stop_words:
        continue
    
    top_domain_specific_words.append(word)
    if len(top_domain_specific_words) == n:
        break
        
all_stop_words = list(string.punctuation) + top_domain_specific_words + stop_words

n = 3
top_vocab = {}
for taster_name in taster_vocabs:
    taster_vocab = taster_vocabs[taster_name]
    
    unique_words = []
    for word, count in taster_vocab.most_common():
        if word in all_stop_words:
            continue
        
        unique_words.append(word)
        if len(unique_words) == n:
            break
        
    top_vocab[taster_name] = unique_words
        
for taster_name, words in top_vocab.items():
    print(f'{taster_name}: ')
    for i, w in enumerate(words):
        print(f'\t{i+1}: {w}')

Let's see what words are most strongly correlated with price

In [ ]:
score_vec = CountVectorizer(
    lowercase=True,
    stop_words=stop_words,
    ngram_range=(1,1),
    min_df=10
)
X_score = score_vec.fit_transform(train_df.description)

In [ ]:
def get_corr(x, y):
    y = np.expand_dims(y, axis=0)
    x = x.toarray().T
    points_np = np.concatenate([x, y], axis=0)
    return np.corrcoef(points_np)[-1][:-1]

In [ ]:
score_corr = get_corr(X_score, train_df.points)

In [ ]:
def display_corr(corr, vec):
    n = 10
    corr_sorted_idx = np.argsort(corr)
    vocab_words = list(vec.vocabulary_.keys())
    
    print('Highest correlations:')
    for i, idx in enumerate(corr_sorted_idx[-1:-n:-1]):
        corr_val = corr[idx]
        word = vocab_words[idx]
        print(f'\t{i+1}. {word}: {corr_val}')

    print('Lowest correlations:')
    for i, idx in enumerate(corr_sorted_idx[:n]):
        corr_val = corr[idx]
        word = vocab_words[idx]
        print(f'\t{i+1}. {word}: {corr_val}')
    
    # plot the correlations
    sorted_score_corr = score_corr[corr_sorted_idx]
    _ = sns.distplot(sorted_score_corr)
    
display_corr(score_corr, score_vec)

Let's do the same thing but for price

In [ ]:
train_price_df = train_df.dropna(subset=['price'], axis=0, inplace=False)

price_vec = CountVectorizer(
    lowercase=True,
    stop_words=stop_words,
    ngram_range=(1,1),
    min_df=10
)
X_price = price_vec.fit_transform(train_price_df.description)
price_corr = get_corr(X_price, train_price_df.price)

In [ ]:
display_corr(price_corr, price_vec)

# NLP Models

## Regression Models

In [ ]:
def rmse(y_true, y_pred):
    return math.sqrt(mse(y_true, y_pred))

def train_and_show(model, x, y, x_dev, y_dev):
    visualizer = PredictionError(model)

    visualizer.fit(x, y)  # Fit the training data to the visualizer
    visualizer.score(x_dev, y_dev)  # Evaluate the model on the test data
    visualizer.poof()
    
    y_pred = visualizer.estimator.predict(x_dev)
    rmse_val = rmse(y_dev, y_pred)
    print(f'Root MSE: {rmse_val:.3f}')
    
X_score_dev = score_vec.transform(dev_df.description)

train_and_show(LinearRegression(), X_score, train_df.points, X_score_dev, dev_df.points)

In [ ]:
train_and_show(XGBRegressor(max_depth=5), X_score, train_df.points, X_score_dev, dev_df.points)

## Classification Models

### By Variety

#### K-Nearest Neighbors 

In [ ]:
# X_train = list(train_df['description'].astype(str))
# y_train = train_df['variety'].astype(str)
# X_dev = list(dev_df['description'].astype(str))
# y_dev = dev_df['variety'].astype(str)
# print(len(X_train), y_train.shape, len(X_dev), y_dev.shape)

In [ ]:
# vectorizer = CountVectorizer()
# train_featurevectors = vectorizer.fit_transform(X_train)
# dev_featurevectors = vectorizer.transform(X_dev)

# #hyperparameter tuning
# param = {'n_neighbors': np.concatenate([np.arange(1,50,1),np.arange(50,100,2),np.arange(100,201,5)]).tolist()}
# knn_bestparam = GridSearchCV(KNeighborsClassifier(), param, scoring='f1_macro')
# knn_bestparam.fit(train_featurevectors, y_train)
# optimal_k = knn_bestparam.best_params_['n_neighbors']

# #model 
# knn = KNeighborsClassifier(n_neighbors=optimal_k)
# knn.fit(train_featurevectors, y_train)
# predictions = knn.predict(dev_featurevectors)
# print('F1 score for a kNN classifier using k=90:', '{0:.4f}'.format(metrics.f1_score(y_dev,predictions, average='macro')))

### By Region

#### K-Nearest Neighbors 

In [ ]:
# y_train = train_df['region'].astype(str)
# y_dev = dev_df['region'].astype(str)
# print(len(X_train), y_train.shape, len(X_dev), y_dev.shape)

In [29]:
# vectorizer = CountVectorizer()
# train_featurevectors = vectorizer.fit_transform(X_train)
# dev_featurevectors = vectorizer.transform(X_dev)

# #hyperparameter tuning
# param = {'n_neighbors': np.concatenate([np.arange(1,50,1),np.arange(50,100,2),np.arange(100,201,5)]).tolist()}
# knn_bestparam = GridSearchCV(KNeighborsClassifier(), param, scoring='f1_macro')
# knn_bestparam.fit(train_featurevectors, y_train)
# optimal_k = knn_bestparam.best_params_['n_neighbors']

# #train the model with optimal hyperparameters
# knn = KNeighborsClassifier(n_neighbors=90)
# knn.fit(train_featurevectors, y_train)
# predictions = knn.predict(dev_featurevectors)
# print("Knn Classification report")
# print('F1 score for a kNN classifier using k=90:', '{0:.4f}'.format(metrics.f1_score(y_dev,predictions, average='macro')))

In [28]:
# The longest description, including start and end tokens
max_len = 130

def surround_texts(texts):
    return [f'SSS {t} EEE' for t in texts]

t = keras.preprocessing.text.Tokenizer()
surrounded_train = surround_texts(train_df.description)
t.fit_on_texts(surrounded_train)

def get_dataset(data, label, batch_size):
    surrounded_series = surround_texts(data)
    seq = t.texts_to_sequences(surrounded_series)
    pad =  keras.preprocessing.sequence.pad_sequences(
        seq, 
        maxlen=max_len,
    )
    label_norm = (label - label.mean()) / label.std()
    
    dataset = tf.data.Dataset.from_tensor_slices((pad, label_norm))
    dataset = dataset.shuffle().batch(batch_size)
    return dataset

train_vec = get_dataset(train_df.description, train_df.points)
dev_vec = get_dataset(dev_df.description, dev_df.points)
test_vec = get_dataset(test_df.description, test_df.points)

RuntimeError: __iter__() is only supported inside of tf.function or when eager execution is enabled.

Meeting Notes:
- Add Price correlation with each word in vocab
  - Make algo more memory efficient
- Establish a baseline
    - Binary Classifier
    - Linear Regression on Highest and Lowest correlation words
        - Price
        - Points
- Do same analysis on Bigrams/Trigrams/NGrams
- Train new models
    - CNN approach
    - LSTM
    - BERT
    - Tasks?
        - Price Regression
        - Score Regression
        - Region Classification
        - Variety Classification
        - Sentiment analysis by Winery
- Appendix
    - Country specific
    - Region specific
    - Taster specific